In [ ]:
#@title AI Paper 音訊分析工具 Colab 快速啟動腳本
#@markdown ---
#@markdown ### 1. 設定與環境準備
#@markdown 此儲存格將會：
#@markdown 1. 掛載您的 Google Drive。
#@markdown 2. 在您的 Google Drive 中建立專案資料夾 (`AI_Paper_Colab_Data`)。
#@markdown 3. 從 GitHub 下載最新的應用程式碼到 Colab 虛擬機。
#@markdown 4. 安裝必要的 Python 套件。
#@markdown 5. 設定環境變數 (包含 API 金鑰和資料夾路徑)。
#@markdown 6. 啟動應用程式並提供公開存取 URL。
#@markdown ---
#@markdown **重要：**
#@markdown - 如果您使用本專案的 **Forked (分支) 版本**，請務必更新下面的 `github_repo_url` 為您自己分支的 URL。
#@markdown - 建議在 Colab 的「密鑰」(Secrets) 中設定 `GOOGLE_API_KEY` (和 `NGROK_AUTHTOKEN`，如果您使用 ngrok)。
#@markdown ---

# 通用設定
# 重要提示：如果您使用的是本儲存庫的 FORK 版本，請將此 URL 更改為您 FORK 的 URL！
github_repo_url = "https://github.com/LaiHao-Alex/AI_paper_audio_analysis.git" #@param {type:"string"}
#@markdown ---
#@markdown ### 2. 執行儲存格
#@markdown 點擊此儲存格左側的執行按鈕 (▶️) 開始設定。
#@markdown 您需要授權 Google Drive 存取權限。
#@markdown 應用程式啟動後，會提供一個 `ngrok.io` 或 `loca.lt` 的公開網址。
#@markdown ---

import os
import sys
import subprocess
from google.colab import drive, output
from IPython.display import display, HTML, Javascript
import threading
import time
import re

# --- 輔助函數 ---
def print_status(message):
    print(f"[*] {message}")

def print_success(message):
    print(f"[成功] {message}")

def print_error(message):
    print(f"[錯誤] {message}")
    # 在 Colab 環境中，sys.exit(1) 可能會導致 Kernel Restart，這裡選擇僅打印錯誤並繼續，讓使用者知曉
    # sys.exit(1) # 如果希望嚴格終止，可以取消註解此行

def run_command(command, description, check=True, capture_output=True):
    print_status(f"執行中: {description}...")
    try:
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE if capture_output else None, stderr=subprocess.PIPE if capture_output else None, text=True)
        stdout, stderr = process.communicate()
        if process.returncode != 0 and check:
            error_message = f"{description} 失敗。"
            if capture_output:
                 error_message += f"\n標準輸出:\n{stdout}\n標準錯誤:\n{stderr}"
            print_error(error_message)
            if check: # 如果 check 為 True，則在此處停止執行後續命令
                 raise subprocess.CalledProcessError(process.returncode, command)
        elif process.returncode != 0 and capture_output:
             print(f"[警告] {description} 可能有非致命錯誤。\n標準輸出:\n{stdout}\n標準錯誤:\n{stderr}")
        else:
            print_success(f"{description} 完成。")
        return stdout, stderr
    except subprocess.CalledProcessError as e:
        # 如果 run_command 設定為 check=True，則異常會在這裡被捕獲並重新拋出，終止腳本的主要流程
        raise e
    except Exception as e:
        print_error(f"執行 '{description}' 時發生例外: {e}")
        if check: # 如果 check 為 True，也重新拋出異常
            raise e


# --- Google Drive 掛載與資料夾設定 ---
try:
    print_status("正在掛載 Google Drive...")
    drive.mount('/content/drive', force_remount=True)

    google_drive_project_root = "/content/drive/MyDrive/AI_Paper_Colab_Data"
    temp_audio_storage_dir_drive = os.path.join(google_drive_project_root, "temp_audio")
    generated_reports_dir_drive = os.path.join(google_drive_project_root, "generated_reports")
    app_code_dir_colab = "/content/app_code" # Colab 虛擬機中應用程式碼的克隆位置

    print_status(f"在 Google Drive 中建立資料夾 (如果不存在):")
    print_status(f"  - 專案根目錄: {google_drive_project_root}")
    os.makedirs(google_drive_project_root, exist_ok=True)
    print_status(f"  - 臨時音訊儲存目錄: {temp_audio_storage_dir_drive}")
    os.makedirs(temp_audio_storage_dir_drive, exist_ok=True)
    print_status(f"  - 生成報告儲存目錄: {generated_reports_dir_drive}")
    os.makedirs(generated_reports_dir_drive, exist_ok=True)
    print_success("Google Drive 資料夾結構設定完成。")

    # --- 應用程式碼克隆/更新 ---
    if os.path.exists(app_code_dir_colab):
        print_status(f"應用程式碼目錄 '{app_code_dir_colab}' 已存在，先移除舊版本...")
        run_command(f"rm -rf {app_code_dir_colab}", "移除舊的應用程式碼目錄")

    print_status(f"從 GitHub ({github_repo_url}) 下載最新的應用程式碼到 Colab 虛擬機 ({app_code_dir_colab})...")
    run_command(f"git clone --depth 1 {github_repo_url} {app_code_dir_colab}", "下載應用程式碼") # 使用 --depth 1 進行淺克隆

    project_root_colab = app_code_dir_colab
    os.chdir(project_root_colab)
    print_success(f"已將工作目錄變更至: {os.getcwd()}")

    # --- 安裝依賴 ---
    requirements_path = os.path.join(project_root_colab, "requirements.txt")
    if not os.path.exists(requirements_path):
        print_error(f"找不到 'requirements.txt' 檔案於: {requirements_path}")
        raise FileNotFoundError(f"找不到 'requirements.txt' 檔案於: {requirements_path}")
    run_command(f"{sys.executable} -m pip install --upgrade pip", "升級 pip")
    run_command(f"{sys.executable} -m pip install -r {requirements_path}", "安裝 Python 套件")

    # --- API 金鑰設定 ---
    print_status("正在設定 Google API 金鑰...")
    google_api_key = ""
    ngrok_auth_token = ""
    try:
        from google.colab import userdata
        google_api_key = userdata.get('GOOGLE_API_KEY')
        ngrok_auth_token = userdata.get('NGROK_AUTHTOKEN') # 也嘗試讀取 ngrok token
        if google_api_key:
            print_success("成功從 Colab Secrets 讀取 GOOGLE_API_KEY。")
        else:
            print_status("Colab Secrets 中未找到 GOOGLE_API_KEY，稍後將提示手動輸入。")
        if ngrok_auth_token:
            print_success("成功從 Colab Secrets 讀取 NGROK_AUTHTOKEN。")
        else:
            print_status("Colab Secrets 中未找到 NGROK_AUTHTOKEN。如果您計劃使用 ngrok 且有 token，建議設定。")

    except ImportError:
        print_status("無法導入 google.colab.userdata (可能為舊版 Colab)，將提示手動輸入金鑰。")
    except Exception as e:
        print_status(f"從 Colab Secrets 讀取金鑰時發生錯誤: {e}。將提示手動輸入。")

    if not google_api_key:
        print_status("請手動輸入您的 Google Gemini API 金鑰:")
        google_api_key = input()
        if google_api_key:
            print_success("已接收手動輸入的 API 金鑰。")
        else:
            print_error("未提供 API 金鑰，應用程式可能無法正常運作。")
            # 在此處可以選擇是否要強制停止，或者讓應用程式嘗試啟動（可能會失敗）
            # raise ValueError("未提供 API 金鑰")

    os.environ['GOOGLE_API_KEY'] = google_api_key
    os.environ['APP_TEMP_AUDIO_STORAGE_DIR'] = temp_audio_storage_dir_drive
    os.environ['APP_GENERATED_REPORTS_DIR'] = generated_reports_dir_drive
    # 如果 app.py 需要知道它在 Colab 環境中運行
    os.environ['RUNNING_IN_COLAB'] = 'true'


    print_success(f"環境變數設定完成。")
    print_status(f"  - GOOGLE_API_KEY: {'已設定 (長度: ' + str(len(google_api_key)) + ')' if google_api_key else '未設定'}")
    print_status(f"  - APP_TEMP_AUDIO_STORAGE_DIR: {os.environ['APP_TEMP_AUDIO_STORAGE_DIR']}")
    print_status(f"  - APP_GENERATED_REPORTS_DIR: {os.environ['APP_GENERATED_REPORTS_DIR']}")

    # --- 啟動伺服器 ---
    print_status("正在準備啟動 FastAPI 應用程式...")
    app_file_path = os.path.join(project_root_colab, "src", "app.py") # 確認路徑正確
    if not os.path.exists(app_file_path):
        print_error(f"找不到應用程式主檔案 'src/app.py' 於: {project_root_colab}")
        raise FileNotFoundError(f"找不到應用程式主檔案 'src/app.py' 於: {project_root_colab}")

    # 使用 threading 管理 ngrok/localtunnel 和 Uvicorn
    def run_uvicorn():
        print_status("正在啟動 Uvicorn 伺服器...")
        # 確保 Uvicorn 從 project_root_colab 運行，以便 src.app 可以被正確找到
        # 使用 sys.executable 確保使用的是 Colab 環境中的 Python 解釋器
        uvicorn_command = [
            sys.executable, "-m", "uvicorn",
            "src.app:app",
            "--host", "0.0.0.0",
            "--port", "8000",
            "--workers", "1"
        ]
        # run_command(uvicorn_command, "啟動 Uvicorn", check=False, capture_output=False) # check=False 因為它是阻塞調用, capture_output=False 讓日誌直接輸出
        # 改為直接 Popen 以更好地控制日誌輸出和錯誤處理
        process = subprocess.Popen(uvicorn_command, stdout=sys.stdout, stderr=sys.stderr, cwd=project_root_colab)
        process.wait() # 等待 Uvicorn 結束

    uvicorn_thread = threading.Thread(target=run_uvicorn)
    uvicorn_thread.daemon = True # 允許主程式退出，即使線程正在運行
    uvicorn_thread.start()
    print_status("Uvicorn 伺服器應已在背景執行緒中啟動。")

    # --- 設定公開 URL (優先使用 ngrok，若失敗則嘗試 localtunnel) ---
    print_status("正在設定公開存取 URL...")
    time.sleep(5) # 給 Uvicorn 一點啟動時間

    public_url = ""
    # Ngrok 設定
    try:
        print_status("嘗試使用 ngrok 建立通道...")
        run_command(f"{sys.executable} -m pip install pyngrok", "安裝/更新 pyngrok")
        from pyngrok import ngrok, conf

        if ngrok_auth_token:
            print_status("使用 Colab Secrets 中的 NGROK_AUTHTOKEN 設定 ngrok。")
            conf.get_default().auth_token = ngrok_auth_token
        else:
            print_status("未在 Colab Secrets 中找到 NGROK_AUTHTOKEN。如果您有 ngrok 帳戶，建議設定以獲得更穩定的服務。")
            # 可以選擇在這裡提示用戶輸入 ngrok token
            # ngrok_token_manual = input("如果您有 ngrok Authtoken，請在此輸入 (否則請留空): ")
            # if ngrok_token_manual:
            #     conf.get_default().auth_token = ngrok_token_manual

        public_url = ngrok.connect(8000)
        print_success(f"Ngrok 通道已建立！")
    except Exception as e_ngrok:
        print_status(f"Ngrok 設定失敗: {e_ngrok}")
        print_status("嘗試備選方案 localtunnel...")
        try:
            # Localtunnel 通常需要全局安裝，如果 Colab 環境限制，可能會有問題
            run_command("npm install -g localtunnel", "安裝 localtunnel (如果尚未安裝)")
            # 使用 Popen 啟動 localtunnel 並捕獲其輸出以獲取 URL
            localtunnel_process = subprocess.Popen(
                f"lt --port 8000", # 可以嘗試添加 --print-requests 來查看更多日誌
                shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, bufsize=1, universal_newlines=True
            )
            # 等待 localtunnel 輸出其 URL，這可能需要幾秒鐘
            # 實時讀取 stdout 來找到 URL
            lt_url_found = False
            for i in range(20): # 嘗試讀取最多 20 秒
                line = localtunnel_process.stdout.readline()
                if line:
                    print_status(f"Localtunnel output: {line.strip()}") # 打印原始輸出以供調試
                    url_match = re.search(r"your url is: (https?://[^\s]+)", line)
                    if url_match:
                        public_url = url_match.group(1)
                        print_success(f"Localtunnel 通道已建立！")
                        lt_url_found = True
                        break
                time.sleep(1)

            if not lt_url_found:
                print_error(f"無法從 localtunnel 輸出中提取 URL。請檢查 localtunnel 是否正確啟動。")
                stdout, stderr = localtunnel_process.communicate() # 獲取剩餘輸出
                if stdout: print_status(f"Localtunnel STDOUT:\n{stdout}")
                if stderr: print_error(f"Localtunnel STDERR:\n{stderr}")
                public_url = "無法建立通道，請檢查上方錯誤訊息。"

        except Exception as e_lt:
            print_error(f"Localtunnel 設定失敗: {e_lt}")
            public_url = "Ngrok 和 Localtunnel 皆設定失敗。"

    print("\n" + "="*50)
    if public_url and "http" in public_url:
        print_success(f"🚀 應用程式應該已經啟動！")
        print(f"🔗 公開存取網址 (Public URL): {public_url}")
        display(HTML(f"<p>點擊此連結開啟應用程式：<a href='{public_url}' target='_blank'>{public_url}</a></p>"))
    else:
        print_error(f"無法生成公開網址。請檢查日誌。")
        display(HTML(f"<p style='color:red;'>無法生成公開網址，請檢查日誌。</p>"))

    print(f"📁 您的資料將會儲存在 Google Drive 的這個位置: {google_drive_project_root}")
    print(f"🕒 您需要保持此 Colab 儲存格持續執行以使用應用程式。")
    print(f"💡 如果遇到問題，請檢查上方儲存格的輸出訊息。")
    print("="*50 + "\n")

    # 保持儲存格運行（可選，因為背景線程可能會使其保持活動狀態）
    # try:
    #     while True:
    #         time.sleep(3600) # 保持活動，偶爾打印狀態
    #         print_status(f"應用程式仍在運行中... 公開網址: {public_url}")
    # except KeyboardInterrupt:
    #     print_status("Colab 執行被使用者中斷。正在關閉...")
    #     if 'ngrok' in sys.modules and public_url and "ngrok.io" in public_url.address:
    #         ngrok.disconnect(public_url.public_url) # 確保傳遞正確的 URL 字符串
    #         ngrok.kill()
    #     # 如何關閉 localtunnel 取決於它是如何啟動的，如果用 Popen，可以嘗試 terminate()
    #     if 'localtunnel_process' in locals() and localtunnel_process.poll() is None:
    #         localtunnel_process.terminate()
    #     print_success("清理完成。")

except Exception as main_exception:
    print_error(f"腳本執行過程中發生未處理的錯誤: {main_exception}")
    # 可以在這裡添加更詳細的錯誤記錄或清理步驟

# 此處的註釋是為了防止 Colab 在腳本結束後自動斷開連接（如果沒有長時間運行的進程）
# 如果 uvicorn_thread.daemon = False，則此儲存格將保持活動狀態直到 Uvicorn 停止或被中斷
# 如果 uvicorn_thread.daemon = True，則主腳本執行完畢後，如果沒有其他前台任務，儲存格可能很快結束
# Ngrok/Localtunnel 的線程或進程是否能保持 Colab 活動取決於 Colab 的策略
# 通常，只要有正在運行的輸出或活動的網絡連接，Colab 會保持運行
# print_status("Colab 腳本主要部分執行完畢。應用程式和通道應在背景運行。")